In [1]:
from time import perf_counter
from typing import Literal, TypedDict

import numpy as np
from astropy.timeseries import LombScargle
from constants import DATA_DIR
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from xgboost import XGBRegressor

from astrofit.model import Asteroid, Lightcurve, LightcurveBin
from astrofit.utils import (
    AsteroidLoader,
    LightcurveBinner,
    LightcurvePlotter,
    LightcurveSplitter,
)

In [2]:
asteroid_loader = AsteroidLoader(DATA_DIR)
lightcurve_binner = LightcurveBinner()
lightcurve_plotter = LightcurvePlotter()
lightcurve_splitter = LightcurveSplitter()

In [3]:
MAX_PERIOD = 13

asteroids: list[Asteroid] = []
for asteroid_name in tqdm(asteroid_loader.available_asteroids):
    asteroid = asteroid_loader.load_asteroid(asteroid_name)
    if asteroid.period > MAX_PERIOD:
        continue

    asteroids.append(asteroid)

print(f"Loaded {len(asteroids)} asteroids")

100%|██████████| 329/329 [00:02<00:00, 114.91it/s]

Loaded 203 asteroids


In [4]:
train_asteroids, test_asteroids = train_test_split(
    asteroids,
    test_size=0.2,
    random_state=88,
)

print(f"Train asteroids: {len(train_asteroids)}")
print(f"Test asteroids: {len(test_asteroids)}")

train_asteroids: list[Asteroid]
test_asteroids: list[Asteroid]

Train asteroids: 162
Test asteroids: 41


In [5]:
class Config(TypedDict):
    max_hours_diff: float
    min_no_points: int
    top_k_bins: int
    select_bins_by: Literal["lightcurves", "points"]
    max_time_diff: float
    min_bin_size: int
    top_k_freqs: int
    nterms: int


In [6]:
def _split_lightcurves(asteroid: Asteroid, config: Config, debug: bool) -> list[Lightcurve]:
    max_hours_diff = config["max_hours_diff"]
    min_no_points = config["min_no_points"]

    if debug:
        print(
            f"Splitting lightcurves for {asteroid.name} with max_hours_diff={max_hours_diff} and min_no_points={min_no_points}"
        )

        print(f"Before splitting: {len(asteroid.lightcurves)} lightcurves")

    splitted = lightcurve_splitter.split_lightcurves(
        asteroid.lightcurves,
        max_hours_diff=max_hours_diff,
        min_no_points=min_no_points,
    )

    if debug:
        print(f"After splitting: {len(splitted)} lightcurves")

    return splitted


def _get_top_k_bins(lightcurves: list[Lightcurve], config: Config, debug: bool) -> list[LightcurveBin]:
    max_time_diff = config["max_time_diff"]
    min_bin_size = config["min_bin_size"]
    top_k_bins = config["top_k_bins"]

    if debug:
        print(f"Getting top {top_k_bins} bins with max_time_diff={max_time_diff} and min_bin_size={min_bin_size}")

    bins = lightcurve_binner.bin_lightcurves(
        lightcurves,
        max_time_diff=max_time_diff,
        min_bin_size=min_bin_size,
    )

    if debug:
        print(f"Got {len(bins)} bins")

    if config["select_bins_by"] == "lightcurves":
        return sorted(bins, reverse=True)[:top_k_bins]

    elif config["select_bins_by"] == "points":
        return sorted(bins, key=lambda bin: bin.points_count, reverse=True)[:top_k_bins]
    
    else:
        raise ValueError("Invalid value for select_bins_by")


def _get_top_k_freqs(lightcurve_bin: LightcurveBin, config: Config, debug: bool):
    nterms = config["nterms"]
    top_k_freqs = config["top_k_freqs"]

    if debug:
        print(
            f"Getting top {top_k_freqs} frequencies with nterms={nterms} for "
            f"lightcurves={len(lightcurve_bin)} with total points={len(lightcurve_bin.times)}"
        )

    frequency, power = LombScargle(
        lightcurve_bin.times,
        lightcurve_bin.brightnesses,
        nterms=nterms,
    ).autopower(maximum_frequency=400)

    idx = np.argsort(power)[::-1][:top_k_freqs]
    top_k_freqs = frequency[idx]

    return np.sort(top_k_freqs)


def get_freq_features(asteroid: Asteroid, config: Config, debug: bool = False) -> np.ndarray | None:
    splitted_lightcurves = _split_lightcurves(asteroid, config, debug)
    top_k_bins = _get_top_k_bins(splitted_lightcurves, config, debug)

    if not top_k_bins or len(top_k_bins) < config["top_k_bins"]:
        if debug:
            print(f"{'-'*50}\n")

        return

    freq_features = []
    for _bin in top_k_bins:
        top_k_freqs = _get_top_k_freqs(_bin, config, debug)
        freq_features.extend(top_k_freqs)

    if debug:
        print(f"{'-'*50}\n")

    return np.array(freq_features)

In [7]:
config = Config(
    max_hours_diff=24,
    min_no_points=10,
    top_k_bins=4,
    select_bins_by="points",
    max_time_diff=45,
    min_bin_size=1,
    top_k_freqs=50,
    nterms=3,
)

In [8]:
failed_cnt = 0

train_X = []
train_y = []
train_processing_times = {}
for asteroid in tqdm(train_asteroids):
    start = perf_counter()
    freq_features = get_freq_features(asteroid, config, debug=True)
    train_processing_times[asteroid.name] = perf_counter() - start

    if freq_features is None:
        failed_cnt += 1
        continue

    train_X.append(freq_features)
    train_y.append(asteroid.period)

train_X = np.vstack(train_X)
train_y = np.array(train_y)

print(f"Failed asteroids: {failed_cnt} ({failed_cnt / len(train_asteroids) * 100:.2f}%)")

train_processing_times = sorted(train_processing_times.items(), key=lambda x: x[1], reverse=True)
for asteroid_name, time in train_processing_times[:10]:
    print(f"{asteroid_name}: {time:.2f}s")

  0%|          | 0/162 [00:00<?, ?it/s]

Splitting lightcurves for Lutetia with max_hours_diff=24 and min_no_points=10
Before splitting: 43 lightcurves
After splitting: 41 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 14 bins
Getting top 50 frequencies with nterms=3 for lightcurves=4 with total points=659
Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=647
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=335
Getting top 50 frequencies with nterms=3 for lightcurves=7 with total points=316


  1%|          | 1/162 [00:09<25:05,  9.35s/it]

--------------------------------------------------

Splitting lightcurves for Euterpe with max_hours_diff=24 and min_no_points=10
Before splitting: 32 lightcurves
After splitting: 38 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 7 bins
Getting top 50 frequencies with nterms=3 for lightcurves=12 with total points=1601
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=1389
Getting top 50 frequencies with nterms=3 for lightcurves=10 with total points=1185
Getting top 50 frequencies with nterms=3 for lightcurves=5 with total points=983


  1%|          | 2/162 [00:19<26:15,  9.85s/it]

...

--------------------------------------------------

Splitting lightcurves for Roberta with max_hours_diff=24 and min_no_points=10
Before splitting: 51 lightcurves
After splitting: 38 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 11 bins
Getting top 50 frequencies with nterms=3 for lightcurves=9 with total points=460
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=435
Getting top 50 frequencies with nterms=3 for lightcurves=5 with total points=340
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=177


 99%|█████████▉| 161/162 [14:05<00:07,  7.24s/it]

--------------------------------------------------

Splitting lightcurves for Bianca with max_hours_diff=24 and min_no_points=10
Before splitting: 49 lightcurves
After splitting: 45 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 10 bins
Getting top 50 frequencies with nterms=3 for lightcurves=8 with total points=430
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=363
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=360
Getting top 50 frequencies with nterms=3 for lightcurves=5 with total points=162


100%|██████████| 162/162 [14:17<00:00,  5.29s/it]

--------------------------------------------------

Failed asteroids: 62 (38.27%)
Eleonora: 16.92s
Thisbe: 16.64s
Ino: 16.28s
Interamnia: 15.96s
Eukrate: 15.85s
Psyche: 15.64s
Vesta: 14.97s
Eros: 14.37s
Nemausa: 14.36s
Siegena: 14.07s


In [9]:
failed_cnt = 0

test_X = []
test_y = []
test_processing_times = {}
for asteroid in tqdm(test_asteroids):
    start = perf_counter()
    freq_features = get_freq_features(asteroid, config, debug=True)
    test_processing_times[asteroid.name] = perf_counter() - start

    if freq_features is None:
        failed_cnt += 1
        continue

    test_X.append(freq_features)
    test_y.append(asteroid.period)

test_X = np.vstack(test_X)
test_y = np.array(test_y)

print(f"Failed asteroids: {failed_cnt} ({failed_cnt / len(test_asteroids) * 100:.2f}%)")

test_processing_times = sorted(test_processing_times.items(), key=lambda x: x[1], reverse=True)
for asteroid_name, time in test_processing_times[:10]:
    print(f"{asteroid_name}: {time:.2f}s")

  0%|          | 0/41 [00:00<?, ?it/s]

Splitting lightcurves for Philomela with max_hours_diff=24 and min_no_points=10
Before splitting: 28 lightcurves
After splitting: 23 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 7 bins
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=387
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=186
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=166
Getting top 50 frequencies with nterms=3 for lightcurves=4 with total points=129


  2%|▏         | 1/41 [00:01<01:18,  1.96s/it]

--------------------------------------------------

Splitting lightcurves for Atalante with max_hours_diff=24 and min_no_points=10
Before splitting: 6 lightcurves
After splitting: 18 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 6 bins
Getting top 50 frequencies with nterms=3 for lightcurves=5 with total points=1540
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=622
Getting top 50 frequencies with nterms=3 for lightcurves=2 with total points=153
Getting top 50 frequencies with nterms=3 for lightcurves=3 with total points=131


  5%|▍         | 2/41 [00:09<03:31,  5.42s/it]

...

Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=144
--------------------------------------------------

Splitting lightcurves for Kassandra with max_hours_diff=24 and min_no_points=10
Before splitting: 14 lightcurves
After splitting: 18 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 7 bins
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=846
Getting top 50 frequencies with nterms=3 for lightcurves=4 with total points=201


 85%|████████▌ | 35/41 [02:30<00:31,  5.21s/it]

Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=122
Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=102
--------------------------------------------------

Splitting lightcurves for Katharina with max_hours_diff=24 and min_no_points=10
Before splitting: 1 lightcurves
After splitting: 4 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 3 bins
Splitting lightcurves for Anna with max_hours_diff=24 and min_no_points=10
Before splitting: 1 lightcurves
After splitting: 0 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 0 bins
Splitting lightcurves for Theresia with max_hours_diff=24 and min_no_points=10
Before splitting: 1 lightcurves
After splitting: 0 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
Got 0 bins
Splitting lightcurves for Dido with max_hours_diff=24 and min_no_points=10
Before splitting: 2 lightcurves
After splitting: 0 lightcurves
Getting top 

100%|██████████| 41/41 [02:35<00:00,  3.80s/it]

Getting top 50 frequencies with nterms=3 for lightcurves=2 with total points=153
--------------------------------------------------

Failed asteroids: 19 (46.34%)
Kalliope: 13.19s
Nysa: 11.66s
Kalypso: 11.24s
Campania: 10.67s
Pandora: 10.62s
Elektra: 10.57s
Bohemia: 10.46s
Athor: 9.97s
Eunomia: 9.31s
Ariadne: 8.05s


In [14]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((100, 200), (100,), (22, 200), (22,))

In [24]:
space = {
    "max_depth": hp.quniform("max_depth", 3, 18, 1),
    "gamma": hp.uniform("gamma", 1, 9),
    "reg_alpha": hp.quniform("reg_alpha", 1, 100, 1),
    "reg_lambda": hp.quniform("reg_lambda", 1, 100, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "min_child_weight": hp.quniform("min_child_weight", 1, 10, 1),
    "n_estimators": hp.quniform("n_estimators", 100, 2000, 1),
    "eta": hp.uniform("eta", 0.01, 0.3),
    "tree_method": hp.choice("tree_method", ["auto", "exact", "approx", "hist"]),
    "max_leaves": hp.quniform("max_leaves", 2, 2**10, 1),
    "max_bin": hp.quniform("max_bin", 2, 2**10, 1),
    "num_parallel_tree": hp.quniform("num_parallel_tree", 1, 10, 1),
    "early_stopping_rounds": hp.quniform("early_stopping_rounds", 10, 100, 1),
    "seed": 88,
}

In [25]:
def objective(space: dict):
    model = XGBRegressor(
        max_depth=int(space["max_depth"]),
        gamma=space["gamma"],      
        reg_alpha=int(space["reg_alpha"]),
        reg_lambda=space["reg_lambda"],
        colsample_bytree=space["colsample_bytree"],
        min_child_weight=int(space["min_child_weight"]),
        n_estimators=int(space["n_estimators"]),
        eta=space["eta"],
        tree_method=space["tree_method"],
        max_leaves=int(space["max_leaves"]),
        max_bin=int(space["max_bin"]),
        num_parallel_tree=int(space["num_parallel_tree"]),
        eval_metric=mean_squared_error,
        early_stopping_rounds=int(space["early_stopping_rounds"]),
        n_jobs=-1,
    )

    evaluation = [(train_X, train_y), (test_X, test_y)]
    model.fit(train_X, train_y, eval_set=evaluation, verbose=False)

    pred_y = model.predict(test_X)
    mse = mean_squared_error(test_y, pred_y)

    return {"loss": mse, "status": STATUS_OK}

In [26]:
trials = Trials()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=1000,
    trials=trials,
)

100%|██████████| 1000/1000 [07:06<00:00,  2.35trial/s, best loss: 1.7138331219191718]


In [32]:
best

{'colsample_bytree': 0.7879339013503472,
 'early_stopping_rounds': 26.0,
 'eta': 0.1415680546588169,
 'gamma': 1.7615108476807284,
 'max_bin': 89.0,
 'max_depth': 15.0,
 'max_leaves': 822.0,
 'min_child_weight': 2.0,
 'n_estimators': 291.0,
 'num_parallel_tree': 4.0,
 'reg_alpha': 3.0,
 'reg_lambda': 1.0,
 'tree_method': 2}

In [34]:
model = XGBRegressor(
    max_depth=int(best["max_depth"]),
    gamma=best["gamma"],      
    reg_alpha=int(best["reg_alpha"]),
    reg_lambda=best["reg_lambda"],
    colsample_bytree=best["colsample_bytree"],
    min_child_weight=int(best["min_child_weight"]),
    n_estimators=int(best["n_estimators"]),
    eta=best["eta"],
    tree_method="exact",
    max_leaves=int(best["max_leaves"]),
    max_bin=int(best["max_bin"]),
    num_parallel_tree=int(best["num_parallel_tree"]),
    eval_metric=mean_squared_error,
    early_stopping_rounds=int(best["early_stopping_rounds"]),
    n_jobs=-1,
)

evaluation = [(train_X, train_y), (test_X, test_y)]
model.fit(train_X, train_y, eval_set=evaluation, verbose=False)

pred_y = model.predict(test_X)
mse = mean_squared_error(test_y, pred_y)

print(f"Mean Squared Error: {mse}\n")

for pred, true in zip(pred_y, test_y):
    print(f"Predicted: {pred:.2f}, True: {true:.2f}")

Mean Squared Error: 2.140780228546522

Predicted: 7.51, True: 8.33
Predicted: 9.60, True: 9.93
Predicted: 9.87, True: 10.74
Predicted: 7.61, True: 7.28
Predicted: 8.51, True: 11.15
Predicted: 5.62, True: 6.08
Predicted: 7.98, True: 8.74
Predicted: 11.68, True: 11.66
Predicted: 7.76, True: 9.82
Predicted: 6.35, True: 5.22
Predicted: 7.30, True: 6.42
Predicted: 6.53, True: 5.66
Predicted: 7.22, True: 4.83
Predicted: 10.61, True: 9.04
Predicted: 7.39, True: 4.80
Predicted: 9.52, True: 7.60
Predicted: 6.82, True: 5.76
Predicted: 7.79, True: 9.45
Predicted: 5.51, True: 4.15
Predicted: 5.75, True: 5.22
Predicted: 8.30, True: 10.74
Predicted: 7.56, True: 7.21
